In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import time
import json
import pickle

In [2]:
#based on the variability of tags, it'll be most efficient to download csvs from AO3 and append the story text 
soup_0005 = BeautifulSoup(open('Fanfic_HTML/0005.html'), "html.parser")
soup_13Heirs = BeautifulSoup(open('Fanfic_HTML/13 Heirs.html'), "html.parser")
soup_WizardComes = BeautifulSoup(open('Fanfic_HTML/In Which A Wizard Comes.html'), "html.parser")

In [3]:
print(len(soup_0005.find(class_="tags")))
print(len(soup_13Heirs.find(class_="tags")))
print(len(soup_WizardComes.find(class_="tags")))

37
37
33


In [11]:
tags = (soup_0005.find(class_="tags"))
tags

<dl class="tags">
<dt>Rating:</dt>
<dd><a href="http://archiveofourown.org/tags/General%20Audiences">General Audiences</a></dd>
<dt>Archive Warning:</dt>
<dd><a href="http://archiveofourown.org/tags/Choose%20Not%20To%20Use%20Archive%20Warnings">Choose Not To Use Archive Warnings</a></dd>
<dt>Category:</dt>
<dd><a href="http://archiveofourown.org/tags/Gen">Gen</a></dd>
<dt>Fandom:</dt>
<dd><a href="http://archiveofourown.org/tags/The%20Umbrella%20Academy%20(TV)">The Umbrella Academy (TV)</a>, <a href="http://archiveofourown.org/tags/The%20Umbrella%20Academy%20(Comics)">The Umbrella Academy (Comics)</a></dd>
<dt>Character:</dt>
<dd><a href="http://archiveofourown.org/tags/Number%20Five%20%7C%20The%20Boy%20(Umbrella%20Academy)">Number Five | The Boy (Umbrella Academy)</a></dd>
<dt>Additional Tags:</dt>
<dd><a href="http://archiveofourown.org/tags/Eating%20Disorders">Eating Disorders</a>, <a href="http://archiveofourown.org/tags/Eating%20Disorder%20Not%20Otherwise%20Specified">Eating Disor

In [14]:
"""def extractTagsClass(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    for element in soup_obj.find(class_="tags"):
        if i == 3:
            story_rating = element.string
        elif i == 7:
            story_archive_warning = element.string
        elif i == 11:
            story_category = element.string
        elif i == 15:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += obj.string + ", "
            fandom_text = fandom_text[:-2]
        elif i == 19:
            story_character = element.string
        elif i == 23:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += obj.string + ", "
            additionalTags_text = additionalTags_text[:-2]
        elif i == 27:
            story_series_name = element.find('a').string
        elif i == 31:
            story_collections = element.string
        elif i == 35:
            pub_stat = str(element.string).strip()
            story_publish_date = pub_stat[11:21]
            story_word_count = pub_stat[37:40]
        i += 1
   
    return story_character, fandom_text, additionalTags_text, story_rating, story_archive_warning, story_category, story_series_name, story_collections, story_publish_date, story_word_count"""

False

Rating:


In [ ]:
print(soup_13Heirs.find(class_="tags"))

In [ ]:
print(soup_WizardComes.find(class_="tags"))

json data = 
Title
AO3 link
Rating
Warning
Audience Category
Fandom(s)
Characters
Tags
Series
Collections
Publish Date
Text Length

In [56]:
def extractFandoms(element):
    #code to extract fandoms
    print(element)
    fandom_text = element.find_all('a')
    print(fandom_text)

    return fandom_text
                
def extractAddTags(element):
    #code to extract tags
    print(element)
    additionalTags = element.find_all('a')
    for obj in additionalTags:
        additionalTags_text += obj.string + ", "
    additionalTags_text = additionalTags_text[:-2]
    print(additionalTags_text)

    return additionalTags_text
    
def extractStoryRating(element):
    #code to extract story rating
    print(element)
    story_rating = element.string
    print(story_rating)
            
    return story_rating
            
def extractArchiveWarning(element):
    #code to extract archive warning
    print(element)
    story_archive_warning = element.string
    print(story_archive_warning)

    return story_archive_warning
            
def extractCategory(element):
    #code to extract category
    print(element)
    story_category = element.string
    print(story_category)

    return story_category
            
def extractCharacters(element):
    #code to extract characters
    print(element)
    story_character = element.string
    print(story_character)
            
    return story_character
            
def extractSeries(element):
    #code to extract series
    print(element)
    story_series_name = element.string
    print(story_series_name)
    
    return story_series_name
    
def extractCollections(element):
    #code to extract collections
    print(element)
    story_collections = element.string
    print(story_collections)
    
    return story_collections

def extractPubStats(element):
    #code to extract publication stats
    print(element)
    pub_stat = str(element.string).strip()
    story_publish_date = pub_stat[11:21]
    story_word_count = pub_stat[37:40]
    print(story_publish_date)
    print(story_word_count)
    
    return story_publish_date, story_word_count

In [58]:
tags_dict = {}

def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    fandom_text = ""
    additionalTags_text = ""
    rating = np.nan
    archiveWarning = np.nan
    category = np.nan
    fandoms = np.nan
    characters = np.nan
    series = np.nan
    collections = np.nan
    pubDate = np.nan
    wordCount = np.nan
    tags = soup_obj.find(class_="tags")
    for element in tags:
        if tags.find(text="Rating:") == "Rating:":
            rating = extractStoryRating(element)
        
        elif tags.find(text="Archive Warning:") == "Archive Warning:":
            archiveWarning = extractArchiveWarning(element)
        
        elif tags.find(text="Category:") == "Category:":
            category = extractCategory(element)
        
        elif tags.find(text="Fandom:") == "Fandom:":
            fandoms = extractFandoms(element)
            
        elif tags.find(text="Character:") == "Character:":
            characters = extractCharacters(element)
            
        elif tags.find(text="Additional Tags:") == "Additional Tags:":
            additionalTags_text = extractAddTags(element)
                
        elif tags.find(text="Series:") == "Series:":
            series = extractSeries(element)
                
        elif tags.find(text="Collections:") == "Collections:":
            collections = extractCollections(element)
            
        elif tags.find(text="Stats:") == "Stats:":
            pubDate, wordCount = extractPubStats(element)
        
        tags_dict["archive_warning"] = archiveWarning
        tags_dict["category"] = category
        tags_dict["fandoms"] = fandoms
        tags_dict["characters"] = characters
        tags_dict["additional_tags"] = additionalTags_text
        tags_dict['rating'] = rating
        tags_dict["series"] = series
        tags_dict["collections"] = collections
        tags_dict["publish_date"] = pubDate 
        tags_dict["word_count"] = wordCount
    
    return tags_dict

In [59]:
tags_dictionary = extractTags(soup_0005)





<dt>Rating:</dt>
Rating:




<dd><a href="http://archiveofourown.org/tags/General%20Audiences">General Audiences</a></dd>
General Audiences




<dt>Archive Warning:</dt>
Archive Warning:




<dd><a href="http://archiveofourown.org/tags/Choose%20Not%20To%20Use%20Archive%20Warnings">Choose Not To Use Archive Warnings</a></dd>
Choose Not To Use Archive Warnings




<dt>Category:</dt>
Category:




<dd><a href="http://archiveofourown.org/tags/Gen">Gen</a></dd>
Gen




<dt>Fandom:</dt>
Fandom:




<dd><a href="http://archiveofourown.org/tags/The%20Umbrella%20Academy%20(TV)">The Umbrella Academy (TV)</a>, <a href="http://archiveofourown.org/tags/The%20Umbrella%20Academy%20(Comics)">The Umbrella Academy (Comics)</a></dd>
None




<dt>Character:</dt>
Character:




<dd><a href="http://archiveofourown.org/tags/Number%20Five%20%7C%20The%20Boy%20(Umbrella%20Academy)">Number Five | The Boy (Umbrella Academy)</a></dd>
Number Five | The Boy (Umbrella Academy)




<dt>Additional Tags:</dt>
Additi

In [60]:
tags_dictionary

{'archive_warning': nan,
 'category': nan,
 'fandoms': nan,
 'characters': nan,
 'additional_tags': '',
 'rating': '\n',
 'series': nan,
 'collections': nan,
 'publish_date': nan,
 'word_count': nan}

In [29]:
character, fandom_text, additionalTags_text, story_rating, story_archive_warning, story_category, story_series_name, story_collections, story_publish_date, story_word_count = extractTags(soup)

In [30]:
story_publish_date

'2019-04-28'

In [31]:
def extractAuthor(soup_obj):
    #code to extract author and fandom info
    for info in soup_obj.title:
        incomplete_title, author, incomplete_fandom = str(info).split('-')
        author = author.strip()
    return author
    

def extractTitleStory(soup_obj):
    #code to extract title and story text
    i = 0
    story_text = ""
    for element in soup_obj.find(id="chapters"):
        if i == 1:
            title = element.string
        elif i == 3:
            story_ptag_text = element.find_all('p')
            for obj in story_ptag_text:
                if obj.string == None:
                    continue
                else:
                    story_text += str(obj.string) + " "
        i += 1
    return story_text
        
def extractTags(soup_obj):
    #code to extract metadata (fandom, characters, tags, warning, etc.)
    i = 0
    fandom_text = ""
    additionalTags_text = ""
    for element in soup_obj.find(class_="tags"):
        if i == 3:
            story_rating = element.string
        elif i == 7:
            story_archive_warning = element.string
        elif i == 11:
            story_category = element.string
        elif i == 15:
            raw_fandom_text = element.find_all('a')
            for obj in raw_fandom_text:
                fandom_text += obj.string + ", "
            fandom_text = fandom_text[:-2]
        elif i == 19:
            story_character = element.string
        elif i == 23:
            additionalTags = element.find_all('a')
            for obj in additionalTags:
                additionalTags_text += obj.string + ", "
            additionalTags_text = additionalTags_text[:-2]
        elif i == 27:
            story_series_name = element.find('a').string
        elif i == 31:
            story_collections = element.string
        elif i == 35:
            pub_stat = str(element.string).strip()
            story_publish_date = pub_stat[11:21]
            story_word_count = pub_stat[37:40]
        i += 1
   
    return story_character, fandom_text, additionalTags_text, story_rating, story_archive_warning, story_category, story_series_name, story_collections, story_publish_date, story_word_count

def extractStoryURL(soup_obj): 
    #code to extract story URL
    i = 0
    for element in soup_obj.find(class_="message"):
        i += 1
        if i == 7:
            url = element['href']

        else:
            continue
    return url

def createAO3dict(filelist):
    i = 0
    ao3_dict = {}
    for element in filelist:
        soup = BeautifulSoup(open(element), "html.parser")
        url = extractStoryURL(soup)
        story_text = extractTitleStory(soup)
        character, fandom_text, additionalTags_text, rating, archive_warning, category, series_name, collections, pub_date, word_count = extractTags(soup)
        author = extractAuthor(soup)
        title = extractTitleStory(soup)
        
        story_dict = {'title': title,
            'author': author,
            'story_text': story_text, 
            'character': character, 
            'url': url
           }
        ao3_dict['i'] = story_dict
        i += 1
    return ao3_dict

In [19]:
fanfics = ['Fanfic_HTML/13 Heirs.html', 'Fanfic_HTML/In Which A Wizard Comes.html', 'Fanfic_HTML/Sweet Basil and Italian.html', 
           'Fanfic_HTML/5 6 7 The Umbrella.html', 'Fanfic_HTML/In Which A Wizard Is.html', 'Fanfic_HTML/Swine and Forests.html',
           'Fanfic_HTML/50 Truths of the.html', 'Fanfic_HTML/In Which Calcifer Meets.html', 'Fanfic_HTML/Symbol of Reform.html',
           'Fanfic_HTML/A Bakers Proposal.html', 'Fanfic_HTML/In Which Howl Attracts.html', 'Fanfic_HTML/Sympathy.html']

In [32]:
ao3_dictionary = createAO3dict(fanfics)

UnboundLocalError: local variable 'story_publish_date' referenced before assignment

In [ ]:
ao3_dictionary

In [140]:
ao3_df = pd.DataFrame.from_dict(ao3_data)
ao3_df

,0
title,00.05
author,verybi_verytired
story_text,He was hungry. Always so hungry. He wasn’t exa...
rating,General Audiences
archive_warning,Choose Not To Use Archive Warnings
category,Gen
fandom_text,"The Umbrella Academy (TV), The Umbrella Academ..."
character,Number Five | The Boy (Umbrella Academy)
additionalTags_text,"Eating Disorders, Eating Disorder Not Otherwis..."
series_name,Angsty Headcanon Ficlets


In [137]:
#code from https://www.geeksforgeeks.org/how-to-convert-python-dictionary-to-json/    
# Serializing json    
json_object = json.dumps(ao3_data, indent = 4)   
print(json_object)

{
    "title": "00.05",
    "author": "verybi_verytired",
    "story_text": "He was hungry. Always so hungry. He wasn\u2019t exactly sure why he was doing this to himself. He didn\u2019t want or need to lose weight, there was always plenty of food, and everyone else was well fed and happy. So why couldn\u2019t he bring himself to eat? During the apocalypse, he barely ate. It was rare that he was full, and when he was full he tended to get sick, his stomach not used to the amount of food in it. Within a year or two of arriving in the ruins of the city, he was used to passing out from a lack of food or water. After he started working as a time travelling assassin he thought it would get better. He was wrong. For some reason, even when literally surrounded by food, he found himself sick at the idea of eating it. Food tasted too good, too strong and it felt weird in his mouth, the textures too much for him to handle. The weight of it in his stomach made him want to curl up or get sick \u20

In [138]:
ao3_df = pd.read_json(json_object, orient='columns')

ValueError: If using all scalar values, you must pass an index

In [114]:
ao3_df

,0
title,00.05
author,verybi_verytired
story_text,He was hungry. Always so hungry. He wasn’t exa...
rating,General Audiences
archive_warning,Choose Not To Use Archive Warnings
category,Gen
fandom_text,"[The Umbrella Academy (TV), The Umbrella Acade..."
character,Number Five | The Boy (Umbrella Academy)
additionalTags_text,"[Eating Disorders, Eating Disorder Not Otherwi..."
series_name,Angsty Headcanon Ficlets


In [116]:
ao3_df.pivot(index=[0])

KeyError: 'None of [None] are in the columns'

In [63]:
print(fandom_text)

['The Umbrella Academy (TV)', 'The Umbrella Academy (Comics)']
